In [56]:
import pandas as pd
import json
from fuzzywuzzy import fuzz
import unidecode as ud
from translate import Translator #pip install translate
translator= Translator(from_lang="es",to_lang="en")
pd.set_option('display.max_colwidth',500)
pd.set_option('display.max_rows', 200)

def partialdecode(s):
    s=str(s)
    return ud.unidecode(s.lower().strip())
def fulldecode(s):
    s=str(s)
    return partialdecode(s
                       ).replace('.',''
                       ).replace(' ',''
                       ).replace('(',''
                       ).replace(')',''
                       ).replace('[',''
                       ).replace(']',''
                       ).replace('{',''
                       ).replace('}',''
                       ).replace('-',''
                       ).replace('_',''
                       ).replace('#',''
                       ).replace('$',''
                       ).replace('\\',''
                       ).replace('%',''
                       ).replace('/',''
                       ).replace('&',''
                       ).replace('"',''
                       ).replace("'",''
                       ).replace("*",''
                       ).replace("^",''
                       ).replace("+",''
                       ).replace("~",''
                       ).replace("`",''
                       ).replace("´",''
                       ).replace(">",''
                       ).replace("<",''
                       ).replace(":",''
                       ).replace(";",''
                       ).replace(",",''
                       ).replace("!",''
                       ).replace("¡",''
                       ).replace("?",''
                       ).replace("¿",''
                       ).replace("|",''
                       )

def quality(row,min_ratio=95,check_ratio=90,min_translation=60,
                                            penalty_partial_ratio=-5,
                                            penalty_token_set_ratio=-5,
                                            penalty_partial_token_set_ratio=-10):
    qr=0;qtst=0;qjr=0
    if not isinstance( row['title_gs'],str):
        return 0
    if not row['title_gs']:
        return 0
    qr=fuzz.ratio(partialdecode(row['title']),partialdecode(row['title_gs']))
    qjr=fuzz.partial_token_set_ratio(partialdecode(row['journal']),partialdecode(row['journal_gs']))
    if qjr<min_ratio:
        qjr=fuzz.partial_token_set_ratio(fulldecode(row['journal']),fulldecode(row['journal_gs']))

    if qr>min_ratio:
        return qr#,'1:'
    #Additional checks requierd from here on
    if qr>=check_ratio:
        if qjr>=min_ratio:
            return qr#,'2:'
        if str(row['volume'])==str(row['volume_gs']):
            return qr
        if str(row['year'])==str(row['year']):
            return qr#,'3:'
    #Partial ratio: one title is included in the other
    #with min_ratio
    qpr=fuzz.partial_ratio(partialdecode(row['title']),partialdecode(row['title_gs']))
    if qpr>=min_ratio:
        if qjr>=min_ratio or str(row['volume'])==str(row['volume_gs']) or str(row['year'])==str(row['year']):
            return qpr+penalty_partial_ratio #,'4:',qpr,
    
    #Token set ratio
    qtser=fuzz.token_set_ratio(partialdecode(row['title']),partialdecode(row['title_gs']))
    #with min_ratio    
    if qtser>=check_ratio:
        if qjr>=min_ratio or str(row['volume'])==str(row['volume_gs']) or str(row['year'])==str(row['year']):
            return qtser+penalty_token_set_ratio#,'5:',qtser,
        
    #with check_ratio    
    if qtser>=check_ratio:
        if (qjr>=min_ratio and 
            str(row['volume'])==str(row['volume_gs']) and
            str(row['year'])==str(row['year'])
           ):
            return qtser+penalty_token_set_ratio#,'6:',qtser,
    #Translations with partial_token_set_ratio
    #with min_ratio
    qptser=fuzz.partial_token_set_ratio(partialdecode(row['title']),partialdecode(row['title_gs']))
    if (qr>=min_translation or qpr>=min_translation or qtser>=min_translation) and qptser>min_ratio:
        if (qjr>=min_ratio and 
            (str(row['volume'])==str(row['volume_gs']) or 
             str(row['year'])==str(row['year'])
           )):
            return max(qr,qpr)+penalty_partial_token_set_ratio#,'7:',max(qr,qpr),
    #with min_translation
    qtsor=fuzz.token_sort_ratio(partialdecode(row['title']),partialdecode(row['title_gs']))
    if (qpr>=min_translation or qtser>=min_translation) and qtsor> min_translation and qptser>min_translation:
        if (qjr>=min_ratio and 
            str(row['volume'])==str(row['volume_gs']) and
            str(row['year'])==str(row['year'])
           ):
            return qpr+penalty_partial_token_set_ratio#,'8:',qpr,
        
        
        
    #Without Translation
    if (qjr>=min_ratio and 
            str(row['volume'])==str(row['volume_gs']) and
            str(row['year'])==str(row['year'])
           ):
        if qr>=min_translation:
            return qr+penalty_partial_token_set_ratio#,'9:',qjr,

    #With Translation!. Only use as last resource
    tpr=0
    if (  qptser>min_translation and 
          qjr>=min_ratio and 
        (str(row['volume'])==str(row['volume_gs']) or 
                 str(row['year'])==str(row['year'])
        )  ):
        s1=translator.translate( str(row['title']) )
        s2=translator.translate( str(row['title_gs']) )
        tpr=fuzz.partial_ratio(partialdecode(s1),partialdecode(s2))
        if tpr>min_translation:
            return tpr+penalty_partial_token_set_ratio#,'10:',tpr
        
    return 0#qr,qpr,qtser,qtsor,qptser,tpr,qjr

In [21]:
with open(r"data/consorcia_unique.json", "r") as read_file:
    data = json.load(read_file)

In [22]:
cs=pd.DataFrame(data)

In [28]:
with open(r"data/scholar-fixes.json", "r") as read_file:
    data = json.load(read_file)

In [29]:
gs=pd.DataFrame(data)

In [31]:
gs.loc[0]

_id                            {'$oid': '5eebebd7b18458a59dccdc3a'}
abstract          El objetivo fue identificar experiencias de im...
article_id                                                         
author            Panader-Torres, Adriana and Cerinza-León, Kare...
bibtex            @article{panader2020experiencias,\n  title={Ex...
cid                                                    D_6K69PKCjsJ
cites                                                           NaN
cites_link                                                      NaN
country                                                            
doi                                          10.21676/2389783X.3234
issue                                                              
journal                                                     Duazary
language                                                           
old_title                                                          
pages                                           

In [32]:
gs.columns

Index(['_id', 'abstract', 'article_id', 'author', 'bibtex', 'cid', 'cites',
       'cites_link', 'country', 'doi', 'issue', 'journal', 'language',
       'old_title', 'pages', 'pdf', 'profiles', 'publisher', 'quality_author',
       'quality_title', 'ref', 'rp', 'timestamp', 'title', 'volume', 'year'],
      dtype='object')

In [33]:
cs.shape

(204536, 5)

In [34]:
cs.loc[0]

doi                                10.1007/s10856-006-0536-5
journal    Journal of Materials Science: Materials in Med...
title      cytotoxicity study of plasma-sprayed hydroxyap...
volume                                                    17
year                                                    2006
Name: 0, dtype: object

In [35]:
gs=gs.rename({'journal':'journal_gs','volume':'volume_gs','year':'year_gs','title':'title_gs'},axis='columns')

In [36]:
gs.columns

Index(['_id', 'abstract', 'article_id', 'author', 'bibtex', 'cid', 'cites',
       'cites_link', 'country', 'doi', 'issue', 'journal_gs', 'language',
       'old_title', 'pages', 'pdf', 'profiles', 'publisher', 'quality_author',
       'quality_title', 'ref', 'rp', 'timestamp', 'title_gs', 'volume_gs',
       'year_gs'],
      dtype='object')

In [37]:
cs.columns

Index(['doi', 'journal', 'title', 'volume', 'year'], dtype='object')

In [38]:
csgs=cs.merge(gs,on='doi',how='left')

## 1) discard exact match

In [39]:
csgs['title_gs'].astype(str)[[18]]

18    nan
Name: title_gs, dtype: object

In [40]:
import unidecode as ud


In [41]:
exact_match=( csgs['title'].apply(lambda s: fulldecode(s) if isinstance(s,str) else s) ==
               csgs['title_gs'].apply(lambda s: fulldecode(s) if isinstance(s,str) else s)
           )
csgsy=csgs[exact_match].reset_index(drop=True)
csgsp=csgs[~exact_match].reset_index(drop=True)

In [ ]:
csgsy['Q']=100

In [42]:
csgsy.shape

(134794, 30)

In [43]:
csgsp.shape

(76248, 30)

In [ ]:
filter_columns=['doi','title','journal','volume','year','title_gs','journal_gs','volume_gs','year_gs']

In [ ]:
csgsp['Q']=csgsp.apply(quality,axis='columns')

## Example translation

In [ ]:
import unidecode as ud
from fuzzywuzzy import fuzz
from translate import Translator #pip install translate
translator= Translator(from_lang="es",to_lang="en")

def partialdecode(s):
    s=str(s)
    return ud.unidecode(s.lower().strip())

s="primer registro del pez vaca cola amarilla hypoplectrus chlorurus (serranidae) en el caribe continental colombiano" 
s1= translator.translate(s)
s='First record of yellowtail hamlet Hypoplectrus chlorurus (Serranidae) in the Colombian continental Caribbean'
s2=translator.translate(s)
s1=partialdecode(s1)
s2=partialdecode(s2)
fuzz.partial_ratio(s1,s2)

## Example `fuzz`

In [697]:
s1='Revista Integración'
s2='Revista Integración, temas de matemáticas'
s1=partialdecode(s1)
s2=partialdecode(s2)

In [698]:
fuzz.ratio(s2,s1)

63

In [699]:
fuzz.partial_ratio(s1,s2)

100

In [700]:
fuzz.token_sort_ratio(s1,s2)

64

In [701]:
fuzz.token_set_ratio(s1,s2)

100

In [702]:
fuzz.partial_token_set_ratio(s1,s2)

100

In [599]:
fuzz.partial_token_sort_ratio(s1,s2)

58